In [1]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=9ff2d8f642bd66050d27630861a78ff5cba271d1da3e8449ee561b2ca3d63025
  Stored in directory: /root/.cache/pip/wheels/4c/6a/a7/bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps


In [2]:
import googlemaps
import pandas as pd
from datetime import datetime
import json
import time
from tqdm import tqdm

### CODIGO PARA OBTENER LA MAYOR CANTIDAD DE RESTAURANTES TOMANDO DIFERENTES PUNTOS DE BOGOTA



In [ ]:
import googlemaps
import json
import time
from datetime import datetime
from tqdm import tqdm

API_KEY = "pongan_su_clave"

gmaps = googlemaps.Client(key=API_KEY)

# ==========
# ZONAS
# ==========
ZONES = {
    "zona_t": (4.6653, -74.0542),
    "zona_g": (4.6483, -74.0591),
    "chapinero": (4.6533, -74.0600),
    "parque_93": (4.6761, -74.0489),
    "usaquen": (4.6905, -74.0306),
    "cedritos": (4.7240, -74.0300),
    "santa_barbara": (4.6980, -74.0374)
}

RADIUS = 3000   # 3 km

# ==========
# AUX
# ==========

def detect_keywords(text, keywords):
    text = text.lower()
    return any(k in text for k in keywords)

def get_reviews(place_id, language="es"):
    details = gmaps.place(
        place_id=place_id,
        fields=["reviews"],
        language=language
    )
    reviews = details.get("result", {}).get("reviews", [])
    return [
        {
            "autor": r.get("author_name"),
            "texto": r.get("text"),
            "rating": r.get("rating"),
            "fecha": datetime.fromtimestamp(r.get("time")).strftime("%Y-%m-%d %H:%M:%S"),
            "idioma": r.get("language")
        }
        for r in reviews if r.get("language", "").startswith("es")
    ]

def get_all_places(location, name):
    all_results = []
    places_result = gmaps.places_nearby(location=location, radius=RADIUS, type='restaurant')

    all_results.extend(places_result.get("results", []))

    while "next_page_token" in places_result:
        token = places_result["next_page_token"]
        time.sleep(2)
        places_result = gmaps.places_nearby(page_token=token)
        all_results.extend(places_result.get("results", []))

    print(f" {name}: {len(all_results)} lugares encontrados")
    return all_results

# ==========
# MAIN
# ==========

def build_bogota_json():
    unique_places = {}
    final_data = []

    print("\n Buscando restaurantes en zonas gastronómicas de Bogotá...\n")

    # 1) Buscar por zona
    for zone_name, coords in ZONES.items():

        print(f"\n Zona: {zone_name}")
        places = get_all_places(coords, zone_name)

        for place in tqdm(places, desc=f"Procesando {zone_name}"):
            place_id = place["place_id"]

            # evitar duplicados
            if place_id in unique_places:
                continue

            details = gmaps.place(
                place_id=place_id,
                fields=[
                    'name','formatted_address','formatted_phone_number','website',
                  'opening_hours','reviews','editorial_summary','rating',
                  'user_ratings_total','geometry','url'
                ],
                language='es'
            )

            result = details.get("result", {})
            if not result:
                continue

            reseñas = get_reviews(place_id)
            if not reseñas:
                continue

            combined_text = " ".join([
                result.get("name", ""),
                result.get("editorial_summary", {}).get("overview", ""),
                " ".join([r["texto"] for r in reseñas])
            ]).lower()

            pet_friendly = detect_keywords(combined_text, ["pet friendly","mascota","perros","gato","aceptan mascotas"])
            vegano       = detect_keywords(combined_text, ["vegano","vegetariano","plant based","sin carne"])

            horario = result.get("opening_hours", {}).get("weekday_text", [])
            abierto_ahora = result.get("opening_hours", {}).get("open_now", None)

            doc = {
                "place_id": place_id,
                "nombre": result.get("name"),
                "direccion": result.get("formatted_address"),
                "telefono": result.get("formatted_phone_number"),
                "website": result.get("website"),
                "rating": result.get("rating"),
                "user_ratings_total": result.get("user_ratings_total"),
                "tipos": result.get("types"),
                "ubicacion": result.get("geometry", {}).get("location"),
                "horarios": horario,
                "abierto_ahora": abierto_ahora,
                "reseñas": reseñas,
                "pet_friendly": pet_friendly,
                "vegano": vegano,
                "url_google_maps": result.get("url"),
                "zona": zone_name,
                "fecha_scraping": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }

            unique_places[place_id] = True
            final_data.append(doc)

            time.sleep(0.3)

    # Guardar
    with open("restaurantes_bogota_multi_zona.json", "w", encoding="utf-8") as f:
        json.dump(final_data, f, ensure_ascii=False, indent=2)

    print(f"\n JSON generado con {len(final_data)} restaurantes únicos.")
    return final_data


if __name__ == "__main__":
    build_bogota_json()



 Buscando restaurantes en zonas gastronómicas de Bogotá...


 Zona: zona_t
 zona_t: 60 lugares encontrados


Procesando zona_t: 100%|██████████| 60/60 [00:16<00:00,  3.65it/s]



 Zona: zona_g
 zona_g: 60 lugares encontrados


Procesando zona_g: 100%|██████████| 60/60 [00:12<00:00,  4.71it/s]



 Zona: chapinero
 chapinero: 60 lugares encontrados


...


 Zona: cedritos
 cedritos: 60 lugares encontrados


Procesando cedritos: 100%|██████████| 60/60 [00:18<00:00,  3.28it/s]



 Zona: santa_barbara
 santa_barbara: 60 lugares encontrados


Procesando santa_barbara: 100%|██████████| 60/60 [00:04<00:00, 12.03it/s]


 JSON generado con 254 restaurantes únicos.


### MODIFICACION PARA TENER LA MAYOR CANTIDAD DE OPINIONES POR CADA RESTAURANTE




In [ ]:
# ============================================
# EXTRACTOR MASIVO DE RESEÑAS GOOGLE PLACES
# ============================================
import googlemaps
import json
import time
from datetime import datetime
from tqdm import tqdm

# ============================================
# CONFIGURACIÓN
# ============================================
API_KEY = "pongan_su_clave"
gmaps = googlemaps.Client(key=API_KEY)

ZONES = {
    "zona_t": (4.6653, -74.0542),
    "zona_g": (4.6483, -74.0591),
    "chapinero": (4.6533, -74.0600),
    "parque_93": (4.6761, -74.0489),
    "usaquen": (4.6905, -74.0306),
    "cedritos": (4.7240, -74.0300),
    "santa_barbara": (4.6980, -74.0374)
}

RADIUS = 3000  # 3 km

RESTAURANTS = [
    'B.O.G. Hotel', 'Hotel Rosales Plaza Bogotá', 'Hotel Chicó 97',
    'Hotel Vilar America', 'Hotel Jazz Apartments', 'The Click Clack Hotel Bogotá',
    'El Sitio By Moris Rodríguez', 'Crepes & Waffles', 'WOK',
    'La Brasa Roja - World Trade Center', 'El Corral', 'Novillera', 'Buena Mesa',
    'La Terrine Delikatessen', 'Habana', 'Teotihuacán Comida Mexicana',
    'El Recreo de Adán', 'Tramonti Restaurante Pizzería', 'Papa Johns Pizza',
    'Restaurante Girassol Parrilla', 'Restaurante La Herencia', 'El Centenario Bogota',
    'La Gran Terraza', 'Restaurante Chambaku', "Pan Pa' Ya! • Retiro",
    'Trattoria Le Calvane', 'Gostinos Centro Internacional', 'DUNKIN',
    'La Ventana restaurante', 'The Wine Store', 'La Plaza de Andrés - El Retiro',
    'Restaurante Japonés Arigato', 'Manhattan Delicatessen', 'Il Pomeriggio',
    'Lina’s Sandwiches', '14 Inkas', 'MiRÁ COMIDA ARGENTINA', 'Taquería Don Clemente',
    'Magnifique El Nogal', 'Archies', 'Bogotá Beer Company', 'El Chato',
    'Frisby', 'Burger King', 'Andrés Exprés', 'Abasto', 'Osaki', 'La Mar • Cebichería Peruana'
]

# ============================================
# FUNCIÓN PRINCIPAL
# ============================================

def get_reviews_for_restaurant(restaurant_name, zones, radius):
    """
    Busca reseñas del restaurante en varias zonas y retorna lista con texto y fecha.
    """
    all_places = {}

    # Buscar el restaurante en cada zona
    for zone_name, location in zones.items():
        try:
            result = gmaps.places_nearby(
                location=location,
                radius=radius,
                keyword=restaurant_name,
                type='restaurant'
            )
            for p in result.get('results', []):
                all_places[p['place_id']] = p
            time.sleep(0.5)
        except Exception as e:
            print(f" Error buscando en {zone_name} para {restaurant_name}: {e}")

    reviews_list = []

    # Obtener reseñas detalladas por lugar encontrado
    for place_id, place in all_places.items():
        try:
            details = gmaps.place(
                place_id=place_id,
                fields=['reviews'],
                language='es'
            )

            for r in details.get('result', {}).get('reviews', []):
                reviews_list.append({
                    "texto": r.get('text', ''),
                    "fecha": datetime.fromtimestamp(r['time']).strftime('%Y-%m-%d'),
                })
            time.sleep(0.5)
        except Exception as e:
            print(f" Error extrayendo reseñas de {place.get('name', 'lugar')}: {e}")
            continue

    # Eliminar duplicados por texto
    unique_reviews = {r["texto"]: r for r in reviews_list if r["texto"].strip()}
    return list(unique_reviews.values())

# ============================================
# PROCESO MASIVO
# ============================================

all_restaurant_reviews = {}

for rest in tqdm(RESTAURANTS, desc="Extrayendo reseñas por restaurante"):
    reviews = get_reviews_for_restaurant(rest, ZONES, RADIUS)
    all_restaurant_reviews[rest] = reviews
    print(f" {rest}: {len(reviews)} reseñas únicas")

# ============================================
# GUARDAR RESULTADOS
# ============================================

output_file = "reseñas_restaurantes_api.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_restaurant_reviews, f, ensure_ascii=False, indent=2)

print(f"\n Archivo guardado en: {output_file}")

# Mostrar resumen
total_reviews = sum(len(v) for v in all_restaurant_reviews.values())
print(f"\n Total de reseñas recopiladas: {total_reviews}")


Extrayendo reseñas por restaurante:   2%|▏         | 1/48 [00:07<05:34,  7.11s/it]

 B.O.G. Hotel: 5 reseñas únicas


Extrayendo reseñas por restaurante:   4%|▍         | 2/48 [00:13<05:11,  6.78s/it]

 Hotel Rosales Plaza Bogotá: 5 reseñas únicas


Extrayendo reseñas por restaurante:   6%|▋         | 3/48 [00:21<05:28,  7.31s/it]

...

 Abasto: 10 reseñas únicas


Extrayendo reseñas por restaurante:  98%|█████████▊| 47/48 [08:38<00:14, 14.34s/it]

 Osaki: 95 reseñas únicas


Extrayendo reseñas por restaurante: 100%|██████████| 48/48 [08:45<00:00, 10.94s/it]

 La Mar • Cebichería Peruana: 5 reseñas únicas

 Archivo guardado en: reseñas_restaurantes_api.json

 Total de reseñas recopiladas: 1820
